In [ ]:
def get_data():
    s1, s2 = [], []
    # processing the anotated ppdb data set from Kevin Gimpel / weitling et al
    with open("ppdb_all.txt", "r") as fp:
        lines = fp.readlines()
        for l in lines:
            l = l.strip()
            dt = l.split("|||")
            if float(dt[2]) < 2.0:
                continue
            s1.append(dt[0])
            s2.append(dt[1])
    return s1, s2

In [2]:
import numpy as np

In [3]:
# load the word to vec and get a vocabulary folder
# create word_vec with glove vectors
word_vec, glove_path = {}, "glove.840B.300d.txt"
with open(glove_path) as f:
    for line in f:
        word, vec = line.split(' ', 1)
        # if word in word_dict:
        word_vec[word] = np.array(list(map(float, vec.split())))
print('Found {0} words with glove vectors'.format(
            len(word_vec)))

Found 2196016 words with glove vectors


In [17]:
import re
def checkword_sent(word_vec, sent):
    sent = sent.replace("!", "").replace("?","")
    for w in sent.split(" "):
        w = w.strip()
        w = w.lower()        
        if w not in word_vec:
            # print('{} not in vocab'.format(w))
            return False
    return True


# getting the 0 indicated data points as negative samples for the training input
def get_negative_data():
    s1, s2 = [], []
    with open("quora_duplicate_questions.tsv", "r") as fp:        
        lines = fp.readlines()
        print(len(lines))
        for i, l in enumerate(lines):
            l = l.strip()
            dt = l.split("\t")
            if i == 0 or len(dt) < 6:
                continue
            
            if not checkword_sent(word_vec, dt[4]):                
                continue
                        
            if not checkword_sent(word_vec, dt[3]):                
                continue
                                    
            if len(dt[3].split(" ")) > 6 or len(dt[4].split(" ")) > 6:
                continue
                
            if len(s1) == 2756:
                break
                
            if int(dt[5]) == 0:
                s1.append(dt[3])
                s2.append(dt[4])
    return s1, s2

In [18]:
sx, sy = get_negative_data()
print(len(sx))
print(len(sy))

404302


2756
2756


In [20]:
for i in zip(sx[:50], sy[:50]):
    print(i)

('What is web application?', 'What is the web application framework?')
('Why do Slavs squat?', 'Will squats make my legs thicker?')
('Nd she is always sad?', 'Aerodynamically what happens when propellor rotates?')
('How do I prevent breast cancer?', 'Is breast cancer preventable?')
('Why did harry become a horcrux?', 'What is a Horcrux?')
('How forgetful are you?', 'How can I forget my wife?')
('Does cognition affect perception?', 'Does cognitive strain affect digestion?')
('Why do Slavs squat?', 'Do squats work for men?')
('How do most people die?', 'How do people die?')
('What is in bone marrow?', 'What does bone marrow taste like?')
('Do you need friends?', 'Why do we need friends?')
('Do I exist?', 'Do we exist?')
('What is journalism for you?', 'What is journalism?')
('How racist is too racist?', 'How racist are you?')
('Are Canada Geese really Canadian?', 'What do Canadians call Canada?')
('How do scientists work?', 'Where do scientists work?')
('What are natural numbers?', 'What

In [21]:
print(checkword_sent(word_vec, "What is the step by step guide to invest in share market in india?"))

True


In [8]:
def get_batch(sents):
    # returns of dimension batch_size * max_len * word_dimension
    # batch is a list of tuples
    lengths = np.array([len(x) for x in sents])
    max_len = np.max(lengths)
    embed = np.zeros((len(sents), max_len, 300))
    for i in range(len(sents)):
        embed[i, :, :] =

2756


In [ ]:
def train():
    s1, s2 = get_data()
    batch_size = 100
    for stdidx in range(0, len(data), batch_size):
        s1 = get_batch(s1[stdidx:stdidx+batch_size])
        s2 = get_batch(s2[stdidx:stdidx+batch_size])